# 0. Setup

### Import Required Libaried


In [1]:
import os
import openai

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

import dotenv 
#load the environment variables of .env file
%load_ext dotenv
%dotenv

### Setup Azure Credentials


In [2]:
# Setup the required credential for using Azure cognitive search
search_endpoint = f"https://{os.getenv('AZURE_SEARCH_SERVICE')}.search.windows.net/"
search_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
search_client = SearchClient(endpoint=search_endpoint,
                                index_name=os.getenv("AZURE_SEARCH_INDEX"),
                                credential=search_creds)

# Setup the required credential for using Azure OpenAI
openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")     
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-05-15"

### Setup Embedding model


In [3]:
def compute_embedding(text):
    return openai.Embedding.create(engine="embedding", input=text)["data"][0]["embedding"]

def nonewlines(s: str) -> str:
    return s.replace(' ', ' ').replace('\r', ' ')

# 1. Using ChatGPT Through calling API

In [8]:
#change the query to what you want to ask chatGPT
query = "Where is the capital of India?"

messages = [
    {'role' : 'user', 'content' : query }
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
chat_content

'The capital of India is New Delhi.'

### Modifying the role of ChatGPT 
- Adding system message
- Adding few shot example

In [9]:
#change the query to what you want to ask chatGPT
query = "Help me write a poem about love."

#change the systemMessage to how you want chatGPT to behave
systemMessage = '''You are a Shakespearean writing assistant who speaks in a Shakespearean style. 
                    You help people come up with creative ideas and content like stories, poems, and songs that use Shakespearean style of writing style, including words like "thou" and "hath”.
                    Here are some example of Shakespeare's style:
                    - Romeo, Romeo! Wherefore art thou Romeo?
                    - Love looks not with the eyes, but with the mind; and therefore is winged Cupid painted blind.
                    - Shall I compare thee to a summer's day? Thou art more lovely and more temperate.'''

messages = [
    {'role' : 'system', 'content' : systemMessage},
    #change the content here to your example question
    {'role' : 'user', 'content' : 'Please write a short text turning down an invitation to dinner.'},
    #change the content here to your example answer
    {'role' : 'assistant', 'content' : '''Dearest,
                                        Regretfully, I must decline thy invitation.
                                        Prior engagements call me hence. Apologies.'''},
    {'role' : 'user', 'content' : query }
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

Of love, the poets oft do write  
With words so sweet, so fair, so bright  
Its power doth move the soul within  
And doth make hearts beat as one.

Love is a flame that doth burn bright  
Its light doth guide us through the night  
It kindles hope within the heart  
And doth make all our fears depart.

Oh, love, how sweet thy power doth flow  
Like a river, it doth ebb and flow  
It doth make the world a brighter place  
And doth bring a smile to every face.

So let us embrace love's gentle might  
And let our hearts take flight  
For in love, we find our truest selves  
And in love, we find our truest wealth.


# 2. Implementing a RAG model

### Obtain related information using vector search

In [12]:
#change the query to what you want to ask the RAG model
query = "Who is the instructor of the course"
query_vector = compute_embedding(query)

r = search_client.search(query, 
                        top=3, 
                        vector=query_vector, 
                        top_k=50, 
                        vector_fields="embedding")

results = [doc["sourcepage"] + ": " + nonewlines(doc["content"]) for doc in r]

for result in results:
    print(result)

The_Innovation_Wings-3.pdf:   In this group, we plan to pursue several interdisciplinary projects that will improve the ways we can study and teach about the human past. Our projects will include uses of augmented and virtual reality in recording, teaching, and presenting archaeological sites. We are also studying the use of machine learning and computer vision for several purposes to study visual datasets such as satellite images of ancient landscapes. We work with a field project that travels to Armenia each summer to excavate. We welcome anyone who would like to join our team, from any Faculty. We especially invite engineers and computer scientists, but we also have many other projects which anyone with an interest can join. Let’s learn about the human past together! Thank you! SIG – HKU Astar (formerly RoboMaster ICRA AI Challenge)  HKU Astar is a student group who participates in Robomaster University AI Challenge Competition. Robomaster University AI Challenge is an AI-based robo

### Query the GPT model with query + information

In [13]:
#change the systemMessage to how you want chatGPT to behave
systemMessage = """AI Assistant that helps user to answer questions from sources provided. Be brief in your answers.
                    Answer ONLY with the facts listed in the list of sources below. 
                    If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. 
                    Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. 
                    Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
                """

messages = [
    {'role' : 'system', 'content' : systemMessage},
    {'role' : 'user', 'content' : query + "   Source:" + " ".join(results)}
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="chat",
    model="gpt-35-turbo",
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

I'm sorry, I don't see any specific information about the instructor of a course in the provided sources.
